<a href="https://colab.research.google.com/github/gt980103/adversarial_examples/blob/master/%E2%80%9CCIFAR10ResNet_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
#使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#设置均值和方差
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)
#设置批量大小
batch_size = 256
lr = 0.01
epochs = 1

#设置数据预处理 
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  #先四周填充0，在吧图像随机裁剪成32*32
    transforms.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
    transforms.ToTensor(),
    transforms.Normalize(mean,std)])

#读取数据集 设置训练集和测试集
train_set = torchvision.datasets.CIFAR10(root='./data',train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./data',train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0)

def evl_test_acc(test_loader ,model, device = None):
    if device is None and isinstance(model, torch.nn.Module):
        # 如果没指定device就使用model的device
        device = list(model.parameters())[0].device 
    with torch.no_grad():
        model.eval()
        total = 0.0
        correct = 0.0
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predict = torch.max(outputs,1)
            total += labels.size(0)
            correct += (labels == predict).sum().item()
        model.train()
    return correct / total

import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import time

model = torchvision.models.resnet50(pretrained=True)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = lr)

start_time = time.time()
each_epoch_time = time.time()

for epoch in range(epochs):
    running_loss = 0.0
    train_total = 0.0
    train_correct = 0.0
    start = time.time()
    for i,data in enumerate(train_loader,0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        #清空梯度
        optimizer.zero_grad()
        #正向传播 反向传播 更新参数
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        #记录损失
        running_loss += loss.item()
        
        #训练集数量 训练集正确数量
        train_total += labels.size(0)
        _, predict = torch.max(output.data,1)
        train_correct += (labels == predict).sum().item()      
        #每50组batch 打印一次
        if i % 50 == 49:
            print('[%d, %5d] loss: %.3f train_acc: %.3f time: %.1f sec' % (epoch + 1, i + 1, running_loss / 50, train_correct / train_total ,time.time()-each_epoch_time))
            running_loss = 0.0
            each_epoch_time = time.time()
    #记录一次epoch的训练误差 测试误差 训练时间
    test_acc = evl_test_acc(test_loader, model)
    print('epoch %d, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch + 1,  train_correct / train_total, test_acc, time.time() - start))
    print("total time %.1f sec" % (time.time() - start_time))

Files already downloaded and verified
Files already downloaded and verified
[1,    50] loss: 3.011 train_acc: 0.129 time: 13.0 sec
[1,   100] loss: 2.415 train_acc: 0.157 time: 13.1 sec
[1,   150] loss: 2.142 train_acc: 0.186 time: 13.2 sec
epoch 1, train acc 0.207, test acc 0.275, time 54.6 sec
total time 54.6 sec
